# H<sub>2</sub><sup>+</sup> Finestructure

In this notebook the hyperfine structure of the H<sub>2</sub><sup>+</sup> molecular ion for a ro-vibrational level ($L$,$\nu$) will be analyzed. Learning the eigenenegies and eigenstates in dependence of the magnetic field $B$.

In [1]:
#import all necessary packages
from IPython.display import display, Markdown, HTML
from H2plus_lib import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact, interactive, Dropdown, FloatLogSlider, FloatSlider, IntText, Checkbox, fixed, IntSlider, Button, interact_manual
import fractions
from H2plus_functions import *


In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

For the analysis of the finestructure we will take into account a Hamiltonian consisting of an effective Breit-Pauli Hamiltonian (hyperfine Hamiltonian) and the Zeeman Hamiltonian. 

$$
H = H_{hfs} + H_Z
$$

The hyperfine Hamiltonian for a rovibrational level is given by:

$$
H_{hfs} = b_F  (\mathbf{I} \cdot \mathbf{S_e}) + c_e  (\mathbf{L} \cdot \mathbf{S_e}) + c_I  (\mathbf{L} \cdot \mathbf{I}) + \frac{d_1}{(2\mathbf{L}-1)(2\mathbf{L}+3)} \left(\frac{2}{3}\mathbf{L}^2(\mathbf{I}\cdot\mathbf{S_e}) - \left\{ (\mathbf{L}\cdot\mathbf{I}),(\mathbf{L}\cdot\mathbf{S_e})\right\}\right) + \frac{d_2}{(2\mathbf{L}-1)(2\mathbf{L}+3)} \left(\frac{1}{3}\mathbf{L}^2\mathbf{I}^2 - \frac{1}{2}(\mathbf{L} \cdot \mathbf{I}) - (\mathbf{L} \cdot \mathbf{I})^2 \right)
$$

where $b_F$, $c_e$, $c_I$, $d_1$ and $d_2$ are  structure constants depending on the ro-vibrational level, which are computed numerically from the non-relativistic Hamiltonian. $\mathbf{I}$ is the total nuclear spin of the two protons, and $\mathbf{S_e}$ is the spin of the electron. This Hamiltonian takes into account realtivistic corrections to the first order in perturbation theory.

The Zeeman Hamiltonian for a magnetic field $\mathbf{B}$ is given by:

$$
H_{Z} = -g_e \mu_B \mathbf{S} \cdot \mathbf{B} - g_{n} \mu_N \mathbf{I} \cdot \mathbf{B} - \mu_B (\mathbf{L}_1 + \mathbf{L}_2) \cdot \mathbf{B} + \mu_B \mathbf{L}_e \cdot \mathbf{B}
$$

where $g_e$ is the g-factor of the electron, $g_{n}$ is the g-factor of the nucleus, $\mu_B$ is the Bohr magneton, $\mu_N$ is the nuclear magneton. Again $\mathbf{S_e}$ is the electron spin, $\mathbf{I}$ is the nuclear spin. $\mathbf{L}_1$ and $\mathbf{L}_2$ are the orbital angular momenta of the two protons, and $\mathbf{L}_e$ is the orbital angular momentum of the electron.

For practical purposes, the basis that we will work with is not the uncoupled $| (\nu) L, m_L, I, m_I, S_e, m_S \rangle$. We introduce the total spin of the system $\mathbf{F}$, which is not an exact quantum number, and the total angular momentum $\mathbf{J}$.

$$
\mathbf{F} = \mathbf{I} + \mathbf{S}_e  \\
\mathbf{J} = \mathbf{L} + \mathbf{F}
$$

This leads to the coupled basis $| ((\nu) L, (I, S_e) F) J, M_J \rangle$. Since $S_e = \frac{1}{2}$ and $I$ is uniquely determined by $L$ for all bound states, if one considers one ro-vibrational level ($\nu$, $L$) we write the basis as $|F,J,M_J \rangle$.

In [3]:
# Define the range of rovibrational levels
L_values = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]  # L values for which coefficients are implemented
nu_values = {0: list(range(0, 4)), 1: list(range(0, 11)), 2: list(range(0, 4)), 3: list(range(0, 11)), 4: list(range(0, 4)), 5: [0], 6: list(range(0, 4)), 7: [0], 8: list(range(0, 4)), 9: [0]}  # nu values for each L

# Create dropdown widgets for L and nu
L_dropdown = Dropdown(options=L_values, description='L:', value=1)
nu_dropdown = Dropdown(options=nu_values[L_dropdown.value], description='$\\nu$:')

display(Markdown('### Select the rovibrational level:'))
display(L_dropdown, nu_dropdown)


### Select the rovibrational level:

Dropdown(description='L:', index=1, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=1)

Dropdown(description='$\\nu$:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10), value=0)

TypeError: update_checkboxes() takes 0 positional arguments but 1 was given

TypeError: update_checkboxes() takes 0 positional arguments but 1 was given

In [4]:
L_ = L_dropdown.value
nu_ = nu_dropdown.value

# Function to update nu_dropdown options based on L_dropdown value
def update_nu_options(*args):
    nu_dropdown.options = nu_values[L_dropdown.value]

# Update nu_dropdown options when L_dropdown value changes
L_dropdown.observe(update_nu_options, 'value')

# Create a logarithmic slider for B_max
B_max_slider = FloatLogSlider(
    value=1e-3,  # initial value
    base=10,  # base of the logarithm
    min=-7,  # minimum exponent of base
    max=1,  # maximum exponent of base
    step=0.2,  # exponent step
    description='$\mathbf{{B}}_{{max}}$:',
    continuous_update=False  # update the value only when the user releases the slider handle
)

# Create an integer input for num_points
num_points_input = IntText(
    value=10000,  # initial value
    description='Points:',
    continuous_update=False  # update the value only when the user finishes editing the text
)


display(Markdown(f'## Breit-Rabi Diagram  \n' \
                 'Plot the different hyperfine energy levels of a rovibrational level ($\\mathbf{{\\nu}},\\mathbf{{L}}$) \
                 against the magnetic field for the selected range. If wanted, one can separate the energy levels\
                that would correspond to the quantum numbers $|\\mathbf{{F}},\\mathbf{{J}}\\rangle$ \
                for the case of $\\mathbf{{B}}=0$ into different plots. \n \
                If desired the derivatives of the energy levels can be plotted as well.'))

# Create a checkbox for separate
separate_checkbox = Checkbox(
    value=False,  # initial value
    description='Separate',
    disabled=False
)

# Create a checkbox for derivatives
derivatives_checkbox = Checkbox(
    value=True,  # initial value
    description='Derivatives',
    disabled=False
)

## Function to link the state of the checkboxes
#def link_checkboxes(change):
#    if change['new']:  # If the Derivatives checkbox is checked
#        separate_checkbox.value = True  # Check the Separate checkbox
#
## Observe the state of the Derivatives checkbox
#derivatives_checkbox.observe(link_checkboxes, names='value')

# Create a dictionary to store the checkboxes and their corresponding indices
checkboxes = {}
FJ_indices = []

# Function to update FJ_indices based on checkbox states
def update_FJ_indices(change):
    if change['new']:  # If the checkbox is checked
        if checkboxes[change['owner']] not in FJ_indices:
            FJ_indices.append(checkboxes[change['owner']])
    else:  # If the checkbox is unchecked
        if checkboxes[change['owner']] in FJ_indices:
            FJ_indices.remove(checkboxes[change['owner']])

# Function to update checkboxes based on L_dropdown value
def update_checkboxes():
    # Get the current checkbox states
    current_states = {i: checkbox.value for checkbox, i in checkboxes.items()}

    # Clear the current checkboxes and FJ_indices
    checkboxes.clear()
    FJ_indices.clear()

    # Get the new FJ_states
    FJ_states = states_FJ(L_)

    # Create checkboxes for each F, J state
    for i, (F, J) in enumerate(FJ_states):
        checkbox = Checkbox(
            value=current_states.get(i, True),  # initial value is the current state if it exists, otherwise True
            description=f'F={F}, J={J}',
            disabled=False
        )
        checkbox.observe(update_FJ_indices, names='value')
        checkboxes[checkbox] = i

    # Display the new checkboxes
    for checkbox in checkboxes:
        display(checkbox)

# Update checkboxes when L_dropdown value changes
L_dropdown.observe(update_checkboxes, 'value')

# Initialize the checkboxes
update_checkboxes()

# Function to wrap around plot_eigenenergies to allow for manual interaction
def plot_eigenenergies_selected_FJ_wrapper(B_max, number_points_B, separate, show_derivatives):
    # Get the current checkbox values
    FJ_indices = [i for checkbox, i in checkboxes.items() if checkbox.value]
    
    # Call plot_eigenenergies_selected_FJ with the current checkbox values
    plot_eigenenergies(nu_, L_, B_max, FJ_indices, number_points_B, separate, show_derivatives)

# Use interact_manual to create a button that calls plot_eigenenergies_selected_FJ_wrapper with the current widget values
interact_manual(plot_eigenenergies_selected_FJ_wrapper, B_max=B_max_slider, number_points_B=num_points_input, separate=separate_checkbox, show_derivatives=derivatives_checkbox);


display(Markdown('# The eigenstates in detail for fixed B field'))


## Breit-Rabi Diagram  
Plot the different hyperfine energy levels of a rovibrational level ($\mathbf{{\nu}},\mathbf{{L}}$)                  against the magnetic field for the selected range. If wanted, one can separate the energy levels                that would correspond to the quantum numbers $|\mathbf{{F}},\mathbf{{J}}\rangle$                 for the case of $\mathbf{{B}}=0$ into different plots. 
                 If desired the derivatives of the energy levels can be plotted as well.

Checkbox(value=True, description='F=1/2, J=1/2')

Checkbox(value=True, description='F=1/2, J=3/2')

Checkbox(value=True, description='F=3/2, J=1/2')

Checkbox(value=True, description='F=3/2, J=3/2')

Checkbox(value=True, description='F=3/2, J=5/2')

interactive(children=(FloatLogSlider(value=0.001, continuous_update=False, description='$\\mathbf{{B}}_{{max}}…

# The eigenstates in detail for fixed B field

In [14]:
L_dropdown_transitions = Dropdown(options=L_values, description='L:', value=1)
nu_dropdown_transitions = Dropdown(options=nu_values[L_dropdown_transitions.value], description='$\\nu$:')

# Create a FloatLogSlider
slider_B = FloatLogSlider(
    value=450e-6,  # initial value
    base=10,  # base of the logarithm
    min=-6,  # min exponent of base
    max=1,  # max exponent of base
    step=0.2,  # exponent step
    description='B'
)

sensitivity_options = [None] + list(range(0, 1501, 100))  # Creates a list with None and numbers from 0 to 1500
sensitivity_dropdown = Dropdown(options=sensitivity_options, description='Sensitivity threshold:')


table_eigenstates(nu_dropdown.value, L_dropdown.value, slider_B.value)

display(Markdown('## Hyperfine transitions at fixed B field'))

# Set max rows displayed to None to display all rows
pd.set_option('display.max_rows', None)

interact(table_hf_transitions, B=slider_B, L=L_dropdown_transitions, nu=nu_dropdown_transitions, prob_treshold=fixed(1e-5), sensitivity_threshold=sensitivity_dropdown)


,0,1,2,3,4,5,6,7,8,9
Column,0,1,2,3,4,5,6,7,8,9
$F$,$\frac{1}{2}$,$\frac{1}{2}$,$\frac{1}{2}$,$\frac{1}{2}$,$\frac{1}{2}$,$\frac{1}{2}$,$\frac{1}{2}$,$\frac{1}{2}$,$\frac{1}{2}$,$\frac{1}{2}$
$J$,$\frac{3}{2}$,$\frac{3}{2}$,$\frac{3}{2}$,$\frac{3}{2}$,$\frac{5}{2}$,$\frac{5}{2}$,$\frac{5}{2}$,$\frac{5}{2}$,$\frac{5}{2}$,$\frac{5}{2}$
$M_J$,$-\frac{3}{2}$,$-\frac{1}{2}$,$\frac{1}{2}$,$\frac{3}{2}$,$-\frac{5}{2}$,$-\frac{3}{2}$,$-\frac{1}{2}$,$\frac{1}{2}$,$\frac{3}{2}$,$\frac{5}{2}$
Color,b,b,b,b,r,r,r,r,r,r
Energy [MHz],-59.715760,-62.352117,-64.860973,-67.259232,35.864216,38.643466,41.273512,43.776058,46.168006,48.462824
Coefficient Squared,0.997361,0.996431,0.996754,0.998017,1.000000,0.997361,0.996431,0.996754,0.998017,1.000000


## Hyperfine transitions at fixed B field

interactive(children=(Dropdown(description='$\\nu$:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10), value=0), Dr…

<function H2plus_functions.table_hf_transitions(nu, L, B, prob_threshold=0.001, sensitivity_threshold=None)>

In [15]:
L_hf_dropdown = Dropdown(options=L_values, description='L:', value=1)
nu_hf_dropdown = Dropdown(options=nu_values[L_hf_dropdown.value], description='$\\nu$:')

B_min_hf = FloatLogSlider(
    value=1e-6,  # initial value
    base=10,  # base of the logarithm
    min=-9,  # minimum exponent of base
    max=1,  # maximum exponent of base
    step=0.2,  # exponent step
    description='$\mathbf{{B}}_{{min}}$:',
    continuous_update=False  # update the value only when the user releases the slider handle
)
B_max_hf = FloatLogSlider(
    value=1e-3,  # initial value
    base=10,  # base of the logarithm
    min=-9,  # minimum exponent of base
    max=1,  # maximum exponent of base
    step=0.2,  # exponent step
    description='$\mathbf{{B}}_{{max}}$:',
    continuous_update=False  # update the value only when the user releases the slider handle
)

interact(table_insensitive_hf_transitions, nu=nu_hf_dropdown, L=L_hf_dropdown,samples=fixed(10000), Bmin=B_min_hf, Bmax=B_max_hf, prob_threshold=fixed(1e-3) , sensitivity_threshold=fixed(100))

interactive(children=(Dropdown(description='$\\nu$:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10), value=0), Dr…

<function H2plus_functions.table_insensitive_hf_transitions(nu, L, Bmin, Bmax, samples=1000, prob_threshold=0.001, sensitivity_threshold=1)>

In [16]:
interact(table_eigenstates, B=slider_B, L=L_dropdown, nu=nu_dropdown)

interactive(children=(Dropdown(description='$\\nu$:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10), value=0), Dr…

<function H2plus_functions.table_eigenstates(nu, L, B)>

In [8]:
display(Markdown('# 2-photon transitions between rovibrational levels'))
# Create dropdown widgets for L and nu
Li_dropdown = Dropdown(options=L_values, description='$L_i$:', value=1)
nu_i_dropdown = Dropdown(options=nu_values[Li_dropdown.value], description='$\\nu_i$:', value=0)
Lf_dropdown = Dropdown(options=L_values, description='$L_f$:', value=1)
nu_f_dropdown = Dropdown(options=nu_values[Lf_dropdown.value], description='$\\nu_f$:', value=1)
B_2photon_slider = FloatLogSlider(
    value=450e-6,  # initial value
    base=10,  # base of the logarithm
    min=-7,  # minimum exponent of base
    max=1,  # maximum exponent of base
    step=0.2,  # exponent step
    description='$\mathbf{{B}}$:',
    continuous_update=False  # update the value only when the user releases the slider handle
)

display(Markdown('### Select the initial rovibrational level:'))
display(nu_i_dropdown,Li_dropdown)
display(Markdown('### Select the final rovibrational level:'))
display(nu_f_dropdown,Lf_dropdown)

# 2-photon transitions between rovibrational levels

### Select the initial rovibrational level:

Dropdown(description='$\\nu_i$:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10), value=0)

Dropdown(description='$L_i$:', index=1, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=1)

### Select the final rovibrational level:

Dropdown(description='$\\nu_f$:', index=1, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10), value=1)

Dropdown(description='$L_f$:', index=1, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=1)

In [19]:
def Two_photon_transitions(B, prob_threshold=1e-7, sensitivity_threshold=None):
    return table_2photon_transitions(nu_i_dropdown.value, Li_dropdown.value, nu_f_dropdown.value, Lf_dropdown.value, B, prob_threshold, sensitivity_threshold)

interact(Two_photon_transitions, B=B_2photon_slider, prob_threshold=fixed(1e-7), sensitivity_threshold=fixed(None))

interactive(children=(FloatLogSlider(value=0.00045, continuous_update=False, description='$\\mathbf{{B}}$:', m…

<function __main__.Two_photon_transitions(B, prob_threshold=1e-07, sensitivity_threshold=None)>

In [10]:
import Beplus as Be

display(Markdown('# 9-Be<sup>+</sup> Finestructure'))



# Define the function to update J options based on L
def update_J_options(*args):
    if Be_L_dropdown.value == 0:
        Be_J_dropdown.options = [0.5]
    else:
        Be_J_dropdown.options = [0.5, 1.5]

# Create a dropdown for L
Be_L_dropdown = Dropdown(options=[0, 1], value=0)

# Create a dropdown for J
Be_J_dropdown = Dropdown(options=[0.5, 1.5], value=0.5)

# Update J options whenever L changes
Be_L_dropdown.observe(update_J_options, 'value')

def plot_and_table(L, J, B_max, B, number_points_B, separate, derivatives):
    display(Markdown('## Breit-Rabi diagrams'))
    Be.plot_eigenenergies(L, J, B_max, number_points_B, separate, derivatives)
    display(Markdown('## Eigenstates in detail for fixed B field'))
    display(Be.table_eigenstates(L, J, B))

# Call interact with the dropdowns
interact(plot_and_table, L=Be_L_dropdown, J=Be_J_dropdown, B_max=B_max_slider, B=slider_B, number_points_B=num_points_input, separate=separate_checkbox, derivatives=derivatives_checkbox)



# 9-Be<sup>+</sup> Finestructure

interactive(children=(Dropdown(description='L', options=(0, 1), value=0), Dropdown(description='J', options=(0…

<function __main__.plot_and_table(L, J, B_max, B, number_points_B, separate, derivatives)>